In [7]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [8]:
# Adapter Layer

def read_csv_to_df(bucket, key, decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files

In [9]:
# Application Layer

def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    df.drop(columns=['prev_closing_price'], inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, trg_key, trg_format):
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, key)
    return True

def etl_report1(src_bucket, trg_bucket, date_list, columns, arg_date, trg_key, trg_format):
    df = extract(src_bucket, date_list)
    df = transform_report1(df, columns, arg_date)
    load(trg_bucket, df, trg_key, trg_format)
    return True
    

In [10]:
# Application Layer - not core

def return_date_list(bucket, arg_date, src_format):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    today = datetime.today().date()
    return_date_list = [(min_date + timedelta(days=x)).strftime(src_format) for x in range(0, (today-min_date).days + 1)]
    return return_date_list

In [11]:
# main function entrypoint

def main():
    # Parameters/Configurations
    # Later read config
    arg_date = '2022-01-28'
    src_format = '%Y-%m-%d'
    src_bucket = 'xetra-1234'
    trg_bucket = 'xetra-1223334444'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report_'
    trg_format = '.parquet'
    
    # Init
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
    
    # run application
    date_list = return_date_list(bucket_src, arg_date, src_format)
    etl_report1(bucket_src, bucket_trg, date_list, columns, arg_date, trg_key, trg_format)

In [12]:
arg_date = '2022-01-28'
src_format = '%Y-%m-%d'
src_bucket = 'xetra-1234'
trg_bucket = 'xetra-1223334444'
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
trg_key = 'xetra_daily_report_'
trg_format = '.parquet'
    
    # Init
s3 = boto3.resource('s3')
bucket_src = s3.Bucket(src_bucket)
bucket_trg = s3.Bucket(trg_bucket)
    
    # run application
date_list = return_date_list(bucket_src, arg_date, src_format)

files = [key for date in date_list for key in list_files_in_prefix(bucket_src, date)]
files

['2022-01-27/2022-01-27_BINS_XETR00.csv',
 '2022-01-27/2022-01-27_BINS_XETR01.csv',
 '2022-01-27/2022-01-27_BINS_XETR02.csv',
 '2022-01-27/2022-01-27_BINS_XETR03.csv',
 '2022-01-27/2022-01-27_BINS_XETR04.csv',
 '2022-01-27/2022-01-27_BINS_XETR05.csv',
 '2022-01-27/2022-01-27_BINS_XETR06.csv',
 '2022-01-27/2022-01-27_BINS_XETR07.csv',
 '2022-01-27/2022-01-27_BINS_XETR08.csv',
 '2022-01-27/2022-01-27_BINS_XETR09.csv',
 '2022-01-27/2022-01-27_BINS_XETR10.csv',
 '2022-01-27/2022-01-27_BINS_XETR11.csv',
 '2022-01-27/2022-01-27_BINS_XETR12.csv',
 '2022-01-27/2022-01-27_BINS_XETR13.csv',
 '2022-01-27/2022-01-27_BINS_XETR14.csv',
 '2022-01-27/2022-01-27_BINS_XETR15.csv',
 '2022-01-27/2022-01-27_BINS_XETR16.csv',
 '2022-01-27/2022-01-27_BINS_XETR17.csv',
 '2022-01-27/2022-01-27_BINS_XETR18.csv',
 '2022-01-27/2022-01-27_BINS_XETR19.csv',
 '2022-01-27/2022-01-27_BINS_XETR20.csv',
 '2022-01-27/2022-01-27_BINS_XETR21.csv',
 '2022-01-27/2022-01-27_BINS_XETR22.csv',
 '2022-01-27/2022-01-27_BINS_XETR2

In [14]:
df = pd.concat([read_csv_to_df(bucket_src, obj) for obj in files], ignore_index=True)
df

C:\Users\user\AppData\Local\Temp\ipykernel_17008\436241567.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([read_csv_to_df(bucket_src, obj) for obj in files], ignore_index=True)


,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,AT0000A0E9W5,SANT,S+T AG O.N.,Common stock,EUR,2504159,2022-01-27,08:00,16.350,16.400,16.350,16.380,7258,4
1,DE000A0D6554,NDX1,NORDEX SE O.N.,Common stock,EUR,2504290,2022-01-27,08:00,13.110,13.200,13.090,13.130,28264,33
2,DE000A0HN5C6,DWNI,DEUTSCHE WOHNEN SE INH,Common stock,EUR,2504314,2022-01-27,08:00,35.390,35.390,35.390,35.390,98,1
3,DE000A0JL9W6,VBK,VERBIO VER.BIOENERGIE ON,Common stock,EUR,2504343,2022-01-27,08:00,50.200,50.250,49.260,49.340,19183,120
4,DE000A0LD2U1,AOX,ALSTRIA OFFICE REIT-AG,Common stock,EUR,2504379,2022-01-27,08:00,19.500,19.500,19.490,19.490,2537,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36409010,GB00BLD4ZP54,CLTC,COINSHARES DIG.SEC.OEND,ETN,EUR,6479084,2022-12-31,16:46,19.324,19.324,19.324,19.324,0,2
36409011,LU1923627332,RUSL,MUL-LYX.MSCI RUSSI.DIS.LS,ETF,EUR,5424594,2022-12-31,16:52,12.400,12.400,12.400,12.400,2645,2
36409012,US98956P1021,ZIM,ZIMMER BIOMET HLDGS DL-01,Common stock,EUR,4582018,2022-12-31,20:30,113.100,113.100,113.100,113.100,0,1
36409013,US9224171002,VEO,"VEECO INSTRUMENTS DL-,01",Common stock,EUR,6198311,2022-12-31,20:30,24.600,24.600,24.600,24.600,0,1


In [26]:
df_small = df.head(5000000)
df_small = transform_report1(df_small, columns, arg_date)
load(bucket_trg, df_small, trg_key, trg_format)


True

In [6]:
# run

main()

ReadTimeoutError: Read timeout on endpoint URL: "None"

## Reading the uploaded file

In [27]:
trg_bucket = 'xetra-1223334444'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)
for obj in bucket_trg.objects.all():
    print(obj.key)

meta_file.csv
xetra_daily_report_20240314_163856.parquet


In [28]:
prq_obj = bucket_trg.Object(key='xetra_daily_report_20240314_163856.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [29]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-01-28,38.05,37.00,37.00,38.05,456,0.00
1,AT000000STR1,2022-01-31,37.80,37.65,37.65,37.80,1492,1.76
2,AT000000STR1,2022-02-01,38.10,38.50,38.10,38.60,668,2.26
3,AT000000STR1,2022-02-02,38.85,38.60,38.55,38.95,2597,0.26
4,AT000000STR1,2022-02-03,39.00,38.60,38.60,39.00,66,0.00
...,...,...,...,...,...,...,...,...
128227,XS2437455608,2022-03-18,24.28,24.71,24.28,24.71,0,1.76
128228,XS2437455608,2022-03-21,24.12,24.12,24.12,24.12,0,-2.40
128229,XS2437455608,2022-03-22,23.96,23.63,23.63,23.96,0,-2.04
128230,XS2437455608,2022-03-23,23.17,23.94,23.17,23.94,0,1.29
